# **Neural network based on the Сovid database**

> Vladyslav Vodopianov

---
Neurónová sieť vytvorená na dátovom súbore covid na výpočet pravdepodobnosti smrti na základe symptómov, veku a pohlavia.

**1.** Najskôr inicializujeme knižnice python.

   * **pandas** - ľahko použiteľný nástroj na analýzu a spracovanie údajov

   * **keras** - knižnica pre vytváranie samotnej neurónovej siete

   * **numpy** - na vektorizáciu, indexáciu a vysielanie

**2.** Načítava sa databáza.

**3.** Vymazanie množiny údajov od nepotrebných údajov a prevod informácií do správnej formy. 
máme 5 stĺpcov ako text. Musíme ich previesť na čísla. Napríklad „**ÁNO**“ - 1, „**NIE**“ - 0. V prípade pohlavia, budeme mať „**Muž**“ - 0, „**Žena**“ - 1.
V prípade veku vytvoríme pole 3 hodnôt. „**Mladý**“ - [ 1, 0, 0], „**Dospelý**“ - [ 0, 1, 0], „**Starší ľudia**“ - [ 0, 0, 1].

**4.** Rozdelenie súboru údajov na účely školenia a testovania.
Na školenie slúži posledných 250 vstupov a výstupov. Najprv 250, použitých na testovanie

**5.**Nastavenie a výcvik neurónovej siete na cvičných dátach.
Inicializujeme neurónovú sieť. Vytvorte 7 vstupov a jeden výstup. Používame metódu relu max(0, z) a metódu sigma. pri tréningu používame v mojom prípade 1 000 epoch.

**6.**Testovanie neurónovej siete na testovacích dátach











Инициализация библиотек

Library initialization

In [104]:
import keras as k
import pandas as pd
import numpy as np

Загрузка базы данных

Loading the database

In [ ]:
data_spis = pd.read_csv("https://raw.githubusercontent.com/vodvlad/datasets/main/covid.csv") 

input_names = ["sex", "age", "cough", "body temperature", "taste and smell"] 
output_names = ["dead"]
#print(data_spis)

Очистка датасета от лишних данных и преобразование информации в правильный вид

Clearing the dataset from unnecessary data and converting information into the correct form

In [ ]:
encoders = {"sex": lambda sex: {"Male": [0], "Female": [1]}.get(sex),
            "age": lambda age: {"Young": [1, 0, 0], "Adult": [0, 1, 0], "Old": [0, 0, 1]}.get(age),
            "cough": lambda cough: {"No": [0], "Yes": [1]}.get(cough),
            "body temperature": lambda body_temperature: {"No": [0], "Yes": [1]}.get(body_temperature),
            "taste and smell": lambda taste_and_smell: {"No": [0], "Yes": [1]}.get(taste_and_smell),
            "dead": lambda dead: [dead]
            }

def dataframe_to_dict(df):
    result = dict()
    for column in df.columns:
        values = data_spis[column].values
        result[column] = values
    return result

def make_supervised(df):
    raw_input_data = data_spis[input_names] # Кладём наши данные из csv файла во входную переменную
    raw_output_data = data_spis[output_names] # Кладём наши данные из csv файла выходную переменную
    return {"inputs": dataframe_to_dict(raw_input_data),
            "outputs": dataframe_to_dict(raw_output_data)}

def encode(data):
    vectors = []
    for data_name, data_values in data.items():
        encoded = list(map(encoders[data_name], data_values))
        vectors.append(encoded)
    formatted = []
    for vector_raw in list(zip(*vectors)):
        vector = []
        for element in vector_raw:
            for e in element:
                vector.append(e)
        formatted.append(vector)

    return formatted


supervised = make_supervised(data_spis)
encoded_inputs = np.array(encode(supervised["inputs"]))
encoded_outputs = np.array(encode(supervised["outputs"]))

Разделение датасета для тренировки и для тестирования

Dividing the dataset for training and testing

In [111]:
train_inputs = encoded_inputs[:250] # использовать для тренировки последние 250 входных чисел
train_outputs = encoded_outputs[:250]

test_inputs = encoded_inputs[250:]  # использовать для теста 250 входных чисел

Настройка и тренировка нейросети на тренировочных данных

Setting up and training a neural network on training data

In [ ]:
model = k.Sequential() # инициализация нейросети
model.add(k.layers.Dense(units=7, activation="relu")) # первый слой, который 7 входных значений
model.add(k.layers.Dense(units=1, activation="sigmoid")) # выходной слой, sigmoid принимает значения от 0 до 1
model.compile(loss="mse", optimizer="sgd", metrics=["accuracy"])
fit_results = model.fit(x=train_inputs, y=train_outputs, epochs=1000, validation_split=0.2) # n эпох для тренировочных входных и выходных чисел.

Тестирование нейросети на тестовых данных

Testing a neural network on test data

In [ ]:
predicted_test = model.predict(test_inputs)
real_data = data_spis.iloc[250:][input_names+output_names]
real_data["dead"] = predicted_test
print(real_data)